In [1]:
from random import shuffle
import glob
import csv
import sys
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [3]:
def loadImage(addr):
    img = cv2.imread(addr)
    if not img is None:
        # Convert it to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [4]:
def createDataRecord(outputAddr, inputAddrs, doAugs):
    writer = tf.io.TFRecordWriter(outputAddr)
    
    for i in range(len(inputAddrs)):
        
        if i % 1000 == 0:
            print("{0} of {1} images completed".format(i, len(inputAddrs)))
        
        img = loadImage(inputAddrs[i])
        
        if img is None:
            print("IMAGE @ " + inputAddrs[i] + " IS NONE, SKIPPING SAMPLE...")
            continue
        
        # This is an augmented sample, alter the expressive face
        if doAugs[i] == 1:
            img = augment(img)
        
        # Resize
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
        
        # Serialize
        feature = {
            'image': _bytes_feature(img.tostring())
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()



In [5]:
def rotateImage(image, angle):
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [6]:
def augment(x: tf.Tensor) -> tf.Tensor:
    whichAug = tf.random.uniform(shape=[1], minval=0, maxval=2, dtype=tf.dtypes.int32)
    if whichAug == 0:
        # Rotate
        rotation = tf.random.uniform(shape=[1], minval=-5, maxval=5, dtype=tf.dtypes.float32)
        x = rotateImage(x, rotation * math.pi / 180)
    elif whichAug == 1:
        # Translate (no more than 15 up or down)
        vertTrans = tf.random.uniform(shape=[1], minval=-9, maxval=10, dtype=tf.dtypes.int32)
        x = x[9+int(vertTrans):246+int(vertTrans), 9:246]
        
    sys.stdout.flush()
    return x

In [9]:
dbBasePath = 'C:\\Users\\Will\\Documents\\StoicNetData\\LFW\\'

with open(dbBasePath + r'images.txt', newline = '') as imageData:
    imageReader = csv.reader(imageData)
    images = [dbBasePath + i[0] for i in imageReader]

# Double the triplets, and label the second half to be altered for augmentation
augs = ([0] * len(images)) + ([1] * len(images))
images = images + list.copy(images)

# Shuffle the triplets (and keep the augmentation labels with them)
c = list(zip(images, augs))
shuffle(c)
images, augs = zip(*c)

# 70/30 train/test split
trainImages = images[0:int(0.7*len(images))]
trainAugs = augs[0:int(0.7*len(augs))]
testImages = images[int(0.7*len(images)):]
testAugs = augs[int(0.7*len(augs)):]


In [10]:
print(images[0])

C:\Users\Will\Documents\StoicNetData\LFW\Mitch_Kupchak\Mitch_Kupchak_0001.jpg


In [11]:
createDataRecord(dbBasePath + 'train.tfrecords', trainImages, trainAugs)
createDataRecord(dbBasePath + 'test.tfrecords', testImages, testAugs)

0 of 18393 images completed
1000 of 18393 images completed
2000 of 18393 images completed
3000 of 18393 images completed
4000 of 18393 images completed
5000 of 18393 images completed
6000 of 18393 images completed
7000 of 18393 images completed
8000 of 18393 images completed
9000 of 18393 images completed
10000 of 18393 images completed
11000 of 18393 images completed
12000 of 18393 images completed
13000 of 18393 images completed
14000 of 18393 images completed
15000 of 18393 images completed
16000 of 18393 images completed
17000 of 18393 images completed
18000 of 18393 images completed
0 of 7883 images completed
1000 of 7883 images completed
2000 of 7883 images completed
3000 of 7883 images completed
4000 of 7883 images completed
5000 of 7883 images completed
6000 of 7883 images completed
7000 of 7883 images completed
